# STA218 Project
## 0.1 data preparation
### dataset introduction
The Data folder contains 2 data files:

1. paper_information.csv    -- the complete list of 5,746 papers including title, publisher, doi, abstract, keywords, references, paper_id  for each  from four journals (the detailed list is shown below). 

2. paper_edge_citation.csv    --the edges in the citation network for the node 'paper_id'. There are 23,737 edges, and each edge represents that 'source' cites 'target' once. Edges between papers are directed. 

Journals are listed as follows:

 [1] JOURNAL OF THE AMERICAN STATISTICAL ASSOCIATION                             
 [2] BIOMETRIKA                                                   
 [3] JOURNAL OF THE ROYAL STATISTICAL SOCIETY SERIES B-STATISTICAL METHODOLOGY  
 [4] ANNALS OF STATISTICS

 We use abbreviation for each journal in the following analysis:  
    [1] JASA                             
    [2] Biometrika                                                  
    [3] JSSR B  
    [4] Ann. Stat

using to find the publishments status and citation network of authors in these four journals. 


In [3]:
import pandas as pd
import sys
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [4]:
DATA_CONFIG = {
    'files': {
        'info': 'paper_information.csv',
        'edges': 'paper_edge_citation.csv'
    },
    'journal_mapping': {
        'JOURNAL OF THE AMERICAN STATISTICAL ASSOCIATION': 'JASA',
        'BIOMETRIKA': 'Biometrika',
        'JOURNAL OF THE ROYAL STATISTICAL SOCIETY SERIES B-STATISTICAL METHODOLOGY': 'JRSS B',
        'ANNALS OF STATISTICS': 'Ann. Stat'
    }
}
info = pd.read_csv(DATA_CONFIG['files']['info'])
edges = pd.read_csv(DATA_CONFIG['files']['edges'])

In [5]:
print(info.shape)
print(edges.shape)

(5746, 7)
(23737, 2)


In [6]:
info.head()

,paper_id,title,publisher,doi,abstract,references,keywords
0,1,MERGING AND TESTING OPINIONS,ANNALS OF STATISTICS,10.1214/14-AOS1212,We study the merging and the testing of opinio...,"no title+Al-Najjar, N.; Pomatto, L.; Sandroni,...","Test manipulation,Bayesian learning"
1,2,EFFICIENT ESTIMATION OF INTEGRATED VOLATILITY ...,ANNALS OF STATISTICS,10.1214/14-AOS1213,We propose new nonparametric estimators of the...,ESTIMATING THE DEGREE OF ACTIVITY OF JUMPS IN ...,"Quadratic variation,Ito semimartingale,integra..."
2,3,FURTHER RESULTS ON CONTROLLING THE FALSE DISCO...,ANNALS OF STATISTICS,10.1214/14-AOS1214,The probability of false discovery proportion ...,The control of the false discovery rate in mul...,"gamma-FDP,generalized gamma-FDP,multiple testi..."
3,4,POSTERIOR CONTRACTION IN SPARSE BAYESIAN FACTO...,ANNALS OF STATISTICS,10.1214/14-AOS1215,Sparse Bayesian factor models are routinely im...,On some inequalities for the incomplete gamma ...,"Bayesian estimation,covariance matrix,factor m..."
4,5,A REMARK ON THE RATES OF CONVERGENCE FOR INTEG...,ANNALS OF STATISTICS,10.1214/13-AOS1179,The optimal rate of convergence of estimators ...,Power and bipower variation withstochastic vol...,"Semimartingale,volatility,jumps,infinite activ..."


In [7]:
edges.head()

,source,target
0,6318,4952
1,3817,4179
2,3817,3783
3,3817,3249
4,3817,6295


In [ ]:
journal_map = DATA_CONFIG['journal_mapping']
info['journal'] = info['publisher'].map(journal_map).fillna(info['publisher'])
info.head()
info.tail

<bound method NDFrame.tail of       paper_id                                              title  \
0            1                       MERGING AND TESTING OPINIONS   
1            2  EFFICIENT ESTIMATION OF INTEGRATED VOLATILITY ...   
2            3  FURTHER RESULTS ON CONTROLLING THE FALSE DISCO...   
3            4  POSTERIOR CONTRACTION IN SPARSE BAYESIAN FACTO...   
4            5  A REMARK ON THE RATES OF CONVERGENCE FOR INTEG...   
...        ...                                                ...   
5741      6502  Parametric and semiparametric models for recap...   
5742      6503  A general method of constructing E(s(2))-optim...   
5743      6506             Dynamic models for spatiotemporal data   
5744      6507  On expected volumes of multidimensional confid...   
5745      6508  Epidemics in heterogeneous communities: estima...   

                                              publisher  \
0                                  ANNALS OF STATISTICS   
1                      

In [ ]:
# --- 2. Functional Core：basic cleaning helpers ---

def clean_title(s: str) -> str:
    """标准化 title：去掉首尾空白、多余空格"""
    if pd.isna(s):
        return ''
    # 把连续空白压成一个空格
    return ' '.join(str(s).strip().split())


def clean_doi(s: str) -> str:
    """标准化 DOI：统一小写、去掉前缀、空值归一"""
    if pd.isna(s) or s in ['NO DOI', '']:
        return ''
    s = str(s).strip()
    s = s.lower()
    # 去掉常见前缀
    for prefix in ['https://doi.org/', 'http://doi.org/', 'doi:']:
        if s.startswith(prefix):
            s = s[len(prefix):]
    return s 